In [1]:
# Importando o Pandas.
import pandas as pd
import pandas_profiling
from collections import Counter #Contador das classes

In [2]:
base_completa = "/home/ubuntu/Geracao_profiling/BaseCompleta.csv"
df = pd.read_csv(base_completa)

/home/ubuntu/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (7,11,12,23,45,46,47,51,53,55,57,63,65,69,75,86,102) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
print(df.shape)
df.head()

(6732542, 120)


,Unnamed: 0,TP_NOT,ID_AGRAVO,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,...,LACO_N,PLASMATICO,EVIDENCIA,PLAQ_MENOR,CON_FHD,COMPLICA,TP_SISTEMA,NDUPLIC_N,CS_FLXRET,FLXRECEBI
0,1,2,A90,2018-05-07,201819,2018,25,251620,1426.0,2504537.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN
1,3,2,A90,2018-02-05,201806,2018,25,250840,1426.0,2606399.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN
2,4,2,A90,2018-01-22,201804,2018,25,250840,1426.0,2606399.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN
3,11,2,A90,2018-05-07,201819,2018,25,251080,1422.0,2605481.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN
4,13,2,A90,2018-04-09,201815,2018,25,251080,1422.0,2605481.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN


In [4]:
df.UF.unique()

array(['PB', 'PE', 'RN', 'AC', 'RO', 'ES', 'GO', 'MS', 'PA', 'AM', 'MG',
       'MT', 'SC', 'SP', 'SE', 'AL', 'TO', 'BA', 'AP', 'CE', 'MA', 'DF',
       'PR', 'RS', 'RJ', 'RR', 'PI'], dtype=object)

In [5]:
df.drop(columns=["Unnamed: 0", "UF"], inplace=True)
df.shape

(6732542, 118)

In [6]:
# df.dropna(subset=['FEBRE'], how='any', inplace = True)
# print(f'Shape após remover linhas com NaN na coluna FEBRE: {df.shape}')
# df.shape

In [7]:
# Confirmação da quantidade de casos de Chikungunya
# Casos de Chikungunya que não estavam marcadas na CLASSI_FIN
df.loc[(df.RES_CHIKS1 == 1) & (df.CLASSI_FIN != 13), 'CLASSI_FIN'] = 13
df[(df.CLASSI_FIN == 13)].shape

(325000, 118)

In [8]:
df = df.loc[:, df.isnull().mean() < .60]
df.shape

(6732542, 60)

In [9]:
df.columns

Index(['TP_NOT', 'ID_AGRAVO', 'DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'SG_UF_NOT',
       'ID_MUNICIP', 'ID_REGIONA', 'ID_UNIDADE', 'DT_SIN_PRI', 'SEM_PRI',
       'DT_NASC', 'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA',
       'CS_ESCOL_N', 'SG_UF', 'ID_MN_RESI', 'ID_RG_RESI', 'ID_PAIS',
       'DT_INVEST', 'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO',
       'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE', 'ARTRALGIA',
       'PETEQUIA_N', 'LEUCOPENIA', 'LACO', 'DOR_RETRO', 'DIABETES',
       'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA', 'ACIDO_PEPT',
       'AUTO_IMUNE', 'RESUL_SORO', 'RESUL_NS1', 'RESUL_VI_N', 'RESUL_PCR_',
       'HISTOPA_N', 'IMUNOH_N', 'HOSPITALIZ', 'TPAUTOCTO', 'COUFINF',
       'COPAISINF', 'COMUNINF', 'CLASSI_FIN', 'CRITERIO', 'EVOLUCAO',
       'DT_ENCERRA', 'TP_SISTEMA', 'CS_FLXRET'],
      dtype='object')

In [10]:
df.drop(columns=["CS_FLXRET", "TP_SISTEMA", "CRITERIO", "TP_NOT"], inplace=True)
df.shape

(6732542, 56)

In [11]:
df.loc[df['CS_SEXO'] == "F", 'CS_SEXO'] = 0
df.loc[df['CS_SEXO']=="M", 'CS_SEXO'] = 1
df.loc[df['CS_SEXO']=="I", 'CS_SEXO'] = 2
df.shape

(6732542, 56)

In [13]:
# COMPLETANDO VALORES NULOS
df.loc[df['CLASSI_FIN'].isnull(), 'CLASSI_FIN'] = 5
# baseCompleta.loc[baseCompleta['CLASSI_FIN']=="", 'CLASSI_FIN'] = 5
# baseCompleta.loc[baseCompleta['CLASSI_FIN']==" ", 'CLASSI_FIN'] = 5
# baseCompleta.loc[baseCompleta['CLASSI_FIN']=='.', 'CLASSI_FIN'] = 5

# DENGUE
# 1 - Dengue Clássico
# 2 - Dengue com Complicações
# 10 - Dengue
# 11 - Dengue com sinais de alarme
# 12 - Dengue grave
df.loc[df['CLASSI_FIN']==1, 'CLASSI_FIN'] = 'Dengue'
df.loc[df['CLASSI_FIN']==2, 'CLASSI_FIN'] = 'Dengue'
df.loc[df['CLASSI_FIN']==10, 'CLASSI_FIN'] = 'Dengue'
df.loc[df['CLASSI_FIN']==11, 'CLASSI_FIN'] = 'Dengue'
df.loc[df['CLASSI_FIN']==12, 'CLASSI_FIN'] = 'Dengue'

# CHIKUNGUNYA
# 13 - Chikungunya
df.loc[df['CLASSI_FIN']==13, 'CLASSI_FIN'] = 'Chikungunya'

# SAUDAVEL
# 5 - Descartado
# 8 - Inconclusivo
# 6 - Outros
df.loc[df['CLASSI_FIN']==5, 'CLASSI_FIN'] = 'Discarded/Inconclusive'
df.loc[df['CLASSI_FIN']==8, 'CLASSI_FIN'] = 'Discarded/Inconclusive'
df.loc[df['CLASSI_FIN']==6, 'CLASSI_FIN'] = 'Discarded/Inconclusive'


#Nos exames, o valor 4 e usado para "nao realizado"
colunas_exames = [
    "RESUL_SORO",
    "RESUL_NS1",
    "RESUL_VI_N",
    "RESUL_PCR_",
    "HISTOPA_N",
    "IMUNOH_N"
]
for coluna in colunas_exames:
    df.loc[df[coluna].isnull(), coluna] = 4

for nome in df.columns:
  df.loc[df[nome].isnull(), nome] = 9
  # baseCompleta.loc[baseCompleta[nome]=="", nome] = 0
  # baseCompleta.loc[baseCompleta[nome]==" ", nome] = 0
  # baseCompleta.loc[baseCompleta[nome]=='.', nome] = 0

# df = df.drop_duplicates()


print(df.shape)
df['CLASSI_FIN'].unique()

(6732542, 56)


array(['DENGUE', 'OUTRAS_DOENCAS', 'CHIKUNGUNYA'], dtype=object)

In [14]:
df.shape

(6732542, 56)

In [15]:
print(Counter(df.CLASSI_FIN))

Counter({'DENGUE': 4307513, 'OUTRAS_DOENCAS': 2100029, 'CHIKUNGUNYA': 325000})


In [16]:
df.columns

Index(['ID_AGRAVO', 'DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'SG_UF_NOT',
       'ID_MUNICIP', 'ID_REGIONA', 'ID_UNIDADE', 'DT_SIN_PRI', 'SEM_PRI',
       'DT_NASC', 'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA',
       'CS_ESCOL_N', 'SG_UF', 'ID_MN_RESI', 'ID_RG_RESI', 'ID_PAIS',
       'DT_INVEST', 'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO',
       'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE', 'ARTRALGIA',
       'PETEQUIA_N', 'LEUCOPENIA', 'LACO', 'DOR_RETRO', 'DIABETES',
       'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA', 'ACIDO_PEPT',
       'AUTO_IMUNE', 'RESUL_SORO', 'RESUL_NS1', 'RESUL_VI_N', 'RESUL_PCR_',
       'HISTOPA_N', 'IMUNOH_N', 'HOSPITALIZ', 'TPAUTOCTO', 'COUFINF',
       'COPAISINF', 'COMUNINF', 'CLASSI_FIN', 'EVOLUCAO', 'DT_ENCERRA'],
      dtype='object')

In [2]:
df = pd.read_csv("/home/ubuntu/Geracao_profiling/BaseCompleta_processada-final.csv", sep=",")
df.head()

/home/ubuntu/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5,9,10,52) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID_AGRAVO,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,DT_SIN_PRI,SEM_PRI,...,HISTOPA_N,IMUNOH_N,HOSPITALIZ,TPAUTOCTO,COUFINF,COPAISINF,COMUNINF,CLASSI_FIN,EVOLUCAO,DT_ENCERRA
0,A90,2018-05-07,201819,2018,25,251620,1426.0,2504537.0,2018-05-06,201819,...,4.0,4.0,9.0,9.0,9.0,9.0,9,DENGUE,1.0,2018-07-01
1,A90,2018-02-05,201806,2018,25,250840,1426.0,2606399.0,2018-02-03,201805,...,4.0,4.0,9.0,9.0,9.0,9.0,9,DENGUE,1.0,2018-03-26
2,A90,2018-01-22,201804,2018,25,250840,1426.0,2606399.0,2018-01-05,201801,...,4.0,4.0,9.0,9.0,9.0,9.0,9,DENGUE,1.0,2018-02-27
3,A90,2018-05-07,201819,2018,25,251080,1422.0,2605481.0,2018-05-04,201818,...,4.0,4.0,2.0,9.0,9.0,9.0,9,OUTRAS_DOENCAS,1.0,2018-07-07
4,A90,2018-04-09,201815,2018,25,251080,1422.0,2605481.0,2018-04-02,201814,...,4.0,4.0,1.0,9.0,9.0,9.0,9,OUTRAS_DOENCAS,1.0,2018-09-04


In [3]:
df.loc[df['ID_AGRAVO']=='A92.', 'ID_AGRAVO'] = 'A92'
df.loc[df['ID_AGRAVO']=='A92.0', 'ID_AGRAVO'] = 'A92'
df.loc[df['ID_AGRAVO']=='A920', 'ID_AGRAVO'] = 'A92'

In [8]:
df.to_csv("/home/ubuntu/Geracao_profiling/BaseCompleta_processada-final.csv", sep=",", index = False)

In [9]:
profile = pandas_profiling.ProfileReport(df, minimal=True, title='Análise dos Dados')
profile.to_file("/home/ubuntu/Geracao_profiling/profiling-final.html")

Summarize dataset:   0%|          | 0/64 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
dengue = df[df.CLASSI_FIN == "Dengue"]
chika = df[df.CLASSI_FIN == "Chikungunya"]
others = df[df.CLASSI_FIN == "Discarded/Inconclusive"]

In [20]:
profile = pandas_profiling.ProfileReport(dengue, minimal=True, title='Análise dos Dados')
profile.to_file("/home/ubuntu/Geracao_profiling/profiling-final-dengue.html")

profile = pandas_profiling.ProfileReport(chika, minimal=True, title='Análise dos Dados')
profile.to_file("/home/ubuntu/Geracao_profiling/profiling-final-chika.html")

profile = pandas_profiling.ProfileReport(others, minimal=True, title='Análise dos Dados')
profile.to_file("/home/ubuntu/Geracao_profiling/profiling-final-others.html")

Summarize dataset:   0%|          | 0/65 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/65 [00:00<?, ?it/s]

...

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/65 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]